In [1]:
import sys
import io
import numpy as np
from requests.auth import HTTPBasicAuth

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import timedelta
import math

from folium import plugins
from folium.plugins import HeatMapWithTime
import folium 

import webbrowser

sys.path.insert(1, '../')
from get_data_API import get_data

In [2]:
from jinja2 import Template 
from folium.map import Layer

class HeatMapWithTimeAdditional(Layer):
    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = new TDHeatmap({{ this.data }},
                {heatmapOptions: {
                    radius: {{this.radius}},
                    minOpacity: {{this.min_opacity}},
                    maxOpacity: {{this.max_opacity}},
                    scaleRadius: {{this.scale_radius}},
                    useLocalExtrema: {{this.use_local_extrema}},
                    defaultWeight: 1,
                    {% if this.gradient %}gradient: {{ this.gradient }}{% endif %}
                }
            }).addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, data, name=None, radius=15,
                 min_opacity=0, max_opacity=0.6,
                 scale_radius=False, gradient=None, use_local_extrema=False,
                 overlay=True, control=True, show=True):
        super(HeatMapWithTimeAdditional, self).__init__(
            name=name, overlay=overlay, control=control, show=show
        )
        self._name = 'HeatMap'
        self.data = data

        # Heatmap settings.
        self.radius = radius
        self.min_opacity = min_opacity
        self.max_opacity = max_opacity
        self.scale_radius = 'true' if scale_radius else 'false'
        self.use_local_extrema = 'true' if use_local_extrema else 'false'
        self.gradient = gradient

In [3]:
# get data
df = get_data()

200


In [4]:
df

,ID,time,temperature,humidity,light,motion,co2,coordinates
0,a8-17-58-ff-fe-03-0f-f7,2023-04-02T00:00:05.127Z,20.7,15,14,2,537,"[65.06146, 25.46666]"
1,a8-17-58-ff-fe-03-0f-5d,2023-04-02T00:00:06.154Z,21.3,12,1,0,516,"[65.06129, 25.4691]"
2,a8-17-58-ff-fe-03-10-91,2023-04-02T00:00:08.866Z,21.3,12,11,0,441,"[65.05849, 25.46598]"
3,a8-17-58-ff-fe-03-0f-ac,2023-04-02T00:00:14.904Z,20.6,15,1,0,539,"[65.05785, 25.46718]"
4,a8-17-58-ff-fe-03-10-8c,2023-04-02T00:00:19.004Z,20.6,16,1,0,516,"[65.05806, 25.46922]"
...,...,...,...,...,...,...,...,...
12788,a8-17-58-ff-fe-03-0f-8a,2023-04-02T11:24:53.344Z,19.9,18,0,1,292,"[65.05692, 25.46841]"
12789,a8-17-58-ff-fe-03-10-74,2023-04-02T11:24:55.769Z,22.2,20,21,0,417,"[65.05973, 25.46816]"
12790,a8-17-58-ff-fe-03-0f-c3,2023-04-02T11:24:57.381Z,22.4,13,59,1,588,"[65.05877, 25.46786]"
12791,a8-17-58-ff-fe-03-0f-ea,2023-04-02T11:24:58.524Z,18.1,15,68,0,525,"[65.05893, 25.46803]"


# Testing function code row by row

In [5]:
selected_data = df[["coordinates", "motion", "time"]]
selected_data

,coordinates,motion,time
0,"[65.06146, 25.46666]",2,2023-04-02T00:00:05.127Z
1,"[65.06129, 25.4691]",0,2023-04-02T00:00:06.154Z
2,"[65.05849, 25.46598]",0,2023-04-02T00:00:08.866Z
3,"[65.05785, 25.46718]",0,2023-04-02T00:00:14.904Z
4,"[65.05806, 25.46922]",0,2023-04-02T00:00:19.004Z
...,...,...,...
12788,"[65.05692, 25.46841]",1,2023-04-02T11:24:53.344Z
12789,"[65.05973, 25.46816]",0,2023-04-02T11:24:55.769Z
12790,"[65.05877, 25.46786]",1,2023-04-02T11:24:57.381Z
12791,"[65.05893, 25.46803]",0,2023-04-02T11:24:58.524Z


In [6]:
#Convert ISO-8601 time format to pandas DateTime format 

selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
selected_data

C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\3334907349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")


,coordinates,motion,time
0,"[65.06146, 25.46666]",2,2023-04-02 00:00:05.127
1,"[65.06129, 25.4691]",0,2023-04-02 00:00:06.154
2,"[65.05849, 25.46598]",0,2023-04-02 00:00:08.866
3,"[65.05785, 25.46718]",0,2023-04-02 00:00:14.904
4,"[65.05806, 25.46922]",0,2023-04-02 00:00:19.004
...,...,...,...
12788,"[65.05692, 25.46841]",1,2023-04-02 11:24:53.344
12789,"[65.05973, 25.46816]",0,2023-04-02 11:24:55.769
12790,"[65.05877, 25.46786]",1,2023-04-02 11:24:57.381
12791,"[65.05893, 25.46803]",0,2023-04-02 11:24:58.524


In [7]:
new = selected_data.groupby(pd.Grouper(key='time', freq='15Min'),as_index=False).apply(lambda x: x['time'])
selected_data['period'] = new.index.get_level_values(0)
selected_data


C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\3887612080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


,coordinates,motion,time,period
0,"[65.06146, 25.46666]",2,2023-04-02 00:00:05.127,0
1,"[65.06129, 25.4691]",0,2023-04-02 00:00:06.154,0
2,"[65.05849, 25.46598]",0,2023-04-02 00:00:08.866,0
3,"[65.05785, 25.46718]",0,2023-04-02 00:00:14.904,0
4,"[65.05806, 25.46922]",0,2023-04-02 00:00:19.004,0
...,...,...,...,...
12788,"[65.05692, 25.46841]",1,2023-04-02 11:24:53.344,45
12789,"[65.05973, 25.46816]",0,2023-04-02 11:24:55.769,45
12790,"[65.05877, 25.46786]",1,2023-04-02 11:24:57.381,45
12791,"[65.05893, 25.46803]",0,2023-04-02 11:24:58.524,45


In [8]:
selected_data['period'].iloc[-1]

45

In [9]:
#create time indexes for HeatMapWithTime
timeIndexes = []
selected_data['period'].iloc[-1]
#get last period in dataframe
for i in range(selected_data['period'].iloc[-1]):
    
    #Add multiples of 15min to first timestamp based on period value
    timeIndexes.append(selected_data["time"].iloc[1] + i * timedelta(minutes=15))
    
len(timeIndexes)

45

In [10]:
#Check amount of data points in each 15 minute segment
df2 = selected_data.pivot_table(index = ['period'], aggfunc ='size')
df2

period
0     292
1     276
2     289
3     288
4     274
5     279
6     293
7     291
8     277
9     277
10    289
11    290
12    284
13    282
14    287
15    290
16    290
17    282
18    284
19    281
20    280
21    273
22    270
23    267
24    284
25    275
26    270
27    281
28    281
29    280
30    274
31    275
32    283
33    270
34    278
35    272
36    277
37    272
38    275
39    274
40    285
41    284
42    278
43    272
44    275
45    193
dtype: int64

# Create function for python file

In [11]:
def data_formating(df, data_shown):
    
    print(f"Formating data for {data_shown} map")
    selected_data = df[["coordinates", data_shown, "time"]]
    
    #Convert ISO-8601 time format to pandas DateTime format 
    selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
    
    #Add new column 'period' to dataframe, every row gets group number based on 15min clusters
    new = selected_data.groupby(pd.Grouper(key='time', freq='15Min'),as_index=False).apply(lambda x: x['time'])
    selected_data['period'] = new.index.get_level_values(0)
    
    #create time indexes for HeatMapWithTime ----------------------
    timeIndexes = []
    
    #get last period in dataframe
    for i in range(selected_data['period'].iloc[-1] + 1):

        #Add multiples of 15min to first timestamp based on period value
        timeIndexes.append((selected_data["time"].iloc[1] + i * timedelta(minutes=15)).strftime("%m/%d/%Y, %H:%M:%S")[:-3])

    #iterate through dataframe -------------------------------------
    data = [[] for _ in range(len(timeIndexes))]
    
    for _, row in selected_data.iterrows():
        #Checking for None type
        if not row[data_shown] is None:
            #Check that data isn't zero or NaN
            if (not math.isnan(row[data_shown]) and row[data_shown] != 0.0):
                
                data[row["period"]].append([row['coordinates'][0],row['coordinates'][1],row[data_shown]])
              
    return data, timeIndexes       
 

In [15]:
#Min-Max scale motion data
column = "motion"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())

#Create heatmap using motion data
motion_data, motion_time = data_formating(df, "motion")


#Create heatmap using co2 data
co2_data, co2_time = data_formating(df, "co2")

#Create heatmap using temperature data
temperature_data, temperature_time = data_formating(df, "temperature")

#Create heatmap using humidity data
humidity_data, humidity_time = data_formating(df, "humidity")


Formating data for motion map


C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


Formating data for co2 map


C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


Formating data for temperature map


C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


Formating data for humidity map


C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Dark\AppData\Local\Temp\ipykernel_7400\1142474530.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


In [16]:
m = folium.Map([65.059228, 25.465375], zoom_start=16.45)

#Motion
HeatMapWithTime(motion_data, index=motion_time, auto_play=True, max_opacity=0.5, name="Motion").add_to(m)
#CO2
HeatMapWithTimeAdditional(co2_data, show=False, max_opacity=0.5, name="CO2").add_to(m)
#Temperature
HeatMapWithTimeAdditional(temperature_data, show=False, max_opacity=0.5, name="temperature").add_to(m)
#Humidity
HeatMapWithTimeAdditional(humidity_data, show=False, max_opacity=0.5, name="humidity").add_to(m)


folium.LayerControl().add_to(m)

m

In [ ]:
folium.LayerControl().add_to(m)
m.save(outfile="index.html")

In [ ]:
#Display html map
webbrowser.open("index.html")